This project is has some objective
* Build a custom model
*	Use different pretrained models(alexnet, vgg16,resnet18, resnet36)
*	Use different optimization algorithm(sgd,sgd with momentum,adam)
*	Use different learning rate.


In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.models as models
from torch.optim import lr_scheduler

In [2]:
#cuda helps in utilizing the GPU for faster computation
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )
device

device(type='cpu')

Now we will import the data

In [3]:
Imagesize = (64,64)
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(Imagesize),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                ])

In [4]:
# dataset = torchvision.datasets.ImageFolder(root='../input/flowers-recognition/flowers/flowers',transform = transform)
# dataset

In [5]:
dataset = torchvision.datasets.ImageFolder(root='data',transform = transform)
dataset

Dataset ImageFolder
    Number of datapoints: 6028
    Root location: data
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )

In [6]:
trainset=torchvision.datasets.ImageFolder(root='data/train',transform = transform)
testset=torchvision.datasets.ImageFolder(root='data/valid',transform = transform)

In [7]:
print("dataset has the following classes ",trainset.classes)
print(type(trainset))
num_classes = len(trainset.classes)
print(num_classes)

dataset has the following classes  ['Autistic', 'Non_Autistic']
<class 'torchvision.datasets.folder.ImageFolder'>
2


* num_workers, which denotes the number of processes that generate batches in parallel. A high enough number of workers assures that CPU computations are efficiently managed, i.e. that the bottleneck is indeed the neural network's forward and backward operations on the GPU (and not data generation).

In [8]:
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=32,num_workers=3, shuffle=True) 
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=32,num_workers=3, shuffle=False)

# Resnet50

In [9]:
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50

C:\Users\doido\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\doido\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
fr = resnet50.fc.in_features
resnet50.fc = nn.Linear(fr, 2)
use_cuda = torch.cuda.is_available()


In [12]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.001)

In [13]:
# 在创建optimizer之后，可以设置一个学习率调度器，例如StepLR
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

## 修改全连接层

In [14]:
TrainLoss = []
TrainAcc = []
TestLoss = []
TestAcc = []
num_epochs = 20

In [15]:
best_accuracy = 0.0

def train(TrainLoss, TrainAcc, TestLoss, TestAcc, num_epochs, model, trainloader, testloader, criterion, optimizer, scheduler=None):
    global best_accuracy
    
    total_step = len(trainloader)
    
    for epoch in range(num_epochs):
        trainAcc = 0
        testAcc = 0

        # Training
        model.train()
        for i, data in enumerate(trainloader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            trainLoss = criterion(outputs, labels)
            trainLoss.backward()
            optimizer.step()

            preds = outputs.argmax(dim=1)
            trainAcc += torch.sum(preds == labels)

        trainAcc = trainAcc.float() / len(trainloader.dataset) * 100

        # Testing
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(testloader):
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                testLoss = criterion(outputs, labels)

                preds = outputs.argmax(dim=1)
                testAcc += torch.sum(preds == labels)

            testAcc = testAcc.float() / len(testloader.dataset) * 100

        # Print and save results
        print("Epoch {} =>  loss : {:.2f};   Accuracy : {:.2f}%;   test_loss : {:.2f};   test_Accuracy : {:.2f}%".format(epoch + 1, trainLoss.item(), trainAcc, testLoss.item(), testAcc))
        
        TrainLoss.append(trainLoss)
        TrainAcc.append(trainAcc)
        TestLoss.append(testLoss)
        TestAcc.append(testAcc)

        # Save best model
        if testAcc > best_accuracy:
            best_accuracy = testAcc
            torch.save(model.state_dict(), 'resnet50.pth')
            print("Best model saved with accuracy: {}%".format(best_accuracy))
        
        # Update learning rate using scheduler
        if scheduler is not None:
            scheduler.step()

In [16]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, resnet50, trainloader, testloader, criterion, optimizer)

Epoch 1 =>  loss : 0.65;   Accuracy : 67.94%;   test_loss : 0.39;   test_Accuracy : 85.00%
Best model saved with accuracy: 85.0%
Epoch 2 =>  loss : 0.50;   Accuracy : 73.59%;   test_loss : 0.32;   test_Accuracy : 86.25%
Best model saved with accuracy: 86.25%
Epoch 3 =>  loss : 0.49;   Accuracy : 78.56%;   test_loss : 0.14;   test_Accuracy : 91.25%
Best model saved with accuracy: 91.25%
Epoch 4 =>  loss : 0.56;   Accuracy : 82.48%;   test_loss : 0.17;   test_Accuracy : 82.50%
Epoch 5 =>  loss : 0.46;   Accuracy : 82.29%;   test_loss : 0.73;   test_Accuracy : 81.25%
Epoch 6 =>  loss : 0.38;   Accuracy : 87.38%;   test_loss : 0.53;   test_Accuracy : 66.25%
Epoch 7 =>  loss : 0.65;   Accuracy : 85.83%;   test_loss : 0.27;   test_Accuracy : 92.50%
Best model saved with accuracy: 92.5%
Epoch 8 =>  loss : 0.48;   Accuracy : 88.43%;   test_loss : 0.19;   test_Accuracy : 88.75%
Epoch 9 =>  loss : 0.30;   Accuracy : 94.50%;   test_loss : 0.24;   test_Accuracy : 91.25%
Epoch 10 =>  loss : 0.14;  